<a href="https://colab.research.google.com/github/lmencisoe/ADL/blob/main/taller%204/Taller4_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_profiling import ProfileReport
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import seaborn as sns
import warnings
import re
warnings.filterwarnings('ignore')

In [3]:
df_1 = pd.read_json('https://raw.githubusercontent.com/lmencisoe/CDA/main/taller%204/DataSet_Entrenamiento_v1.json')
df_2 = pd.read_json('https://raw.githubusercontent.com/lmencisoe/CDA/main/taller%204/DataSet_Entrenamiento_v2.json')

In [129]:
df_1['TotalCharges'] = df_1['TotalCharges'].replace("", 0)
df_1['TotalCharges'] = df_1['TotalCharges'].astype(float)
df_2['TotalCharges'] = df_2['TotalCharges'].replace("", 0)
df_2['TotalCharges'] = df_2['TotalCharges'].astype(float)

In [149]:
X_total = df_1.drop(['Churn', 'customerID'], axis=1)
df_1['target'] = np.where(df_1['Churn']!= 'Yes', 1, 0)
Y_total = df_1['target'].astype(float)
X_total = X_total.drop(['target'], axis=1)

In [150]:
X_total2 = df_2.drop(['Churn', 'customerID'], axis=1)
df_2['target'] = np.where(df_2['Churn']!= 'Yes', 1, 0)
Y_total2 = df_2['target'].astype(float)
X_total2 = X_total2.drop(['target'], axis=1)

In [151]:
cat_features = X_total.select_dtypes(exclude=["float64", "int64"]).columns.to_list()
numeric_features = X_total.select_dtypes(["float64", "int64"]).columns.to_list()

categorical_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('encoder', OneHotEncoder())
            ]
        )    

numeric_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())
            ]
        )

preprocessor = ColumnTransformer(
            transformers=[
                ('numerical', numeric_transformer, numeric_features),
                ('categorical', categorical_transformer, cat_features)
            ]
        )

In [152]:
consideration_pipeline = Pipeline(
            [
                ("preprocessor", preprocessor),
                (
                    "classifier",
                    XGBClassifier(),
                ),
            ]
        )

fit_pipe = consideration_pipeline.fit(X_total, Y_total)

In [153]:
cross_val_scores = cross_val_score(consideration_pipeline, X_total, Y_total, scoring='roc_auc', cv=10)

In [154]:
consideration_pipeline.score(X_total, Y_total)

0.8206666666666667

In [155]:
X_total2.shape

(1000, 19)

In [156]:
X_total.shape

(6000, 19)

In [157]:
Y_total.sum()

4408.0

In [158]:
Y_total2.sum()

735.0

In [159]:
fit_pipe2 = consideration_pipeline.fit(X_total2, Y_total2)

In [160]:
consideration_pipeline.score(X_total2, Y_total2)

0.865

In [148]:
X_total

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,target
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,1
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,1
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,0
3,Female,1,Yes,No,66,No,No phone service,DSL,No,Yes,Yes,No,Yes,No,One year,No,Bank transfer (automatic),45.55,3027.25,1
4,Male,0,Yes,No,68,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),110.00,7611.85,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Female,0,Yes,No,40,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.10,780.10,1
5996,Male,0,Yes,Yes,6,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Credit card (automatic),20.40,107.60,1
5997,Male,0,No,No,39,Yes,No,Fiber optic,Yes,No,Yes,No,Yes,Yes,Two year,Yes,Electronic check,100.45,3801.70,1
5998,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,No,Electronic check,74.95,308.70,0
